In [1]:
from hypergraphs import *
from kegg import *

In [3]:
superpathwayDict = {
    "Carbohydrate metabolism":
[
	("00010", "Glycolysis / Gluconeogenesis"),
	("00020", "Citrate cycle (TCA cycle)"),
	("00030", "Pentose phosphate pathway"),
	("00040", "Pentose and glucuronate interconversions"),
	("00051", "Fructose and mannose metabolism"),
	("00052", "Galactose metabolism"),
	("00053", "Ascorbate and aldarate metabolism"),
	("00500", "Starch and sucrose metabolism"),
	("00520", "Amino sugar and nucleotide sugar metabolism"),
	("00620", "Pyruvate metabolism"),
	("00630", "Glyoxylate and dicarboxylate metaboliscm"),
	("00640", "Propanoate metabolism"),
	("00650", "Butanoate metabolism"),
	("00562", "Inositol phosphate metabolism")
],
"Energy metabolism":
[
	("00190", "Oxidative phosphorylation"),
	("00910", "Nitrogen metabolism"),
	("00920", "Sulfur metabolism")
],
",Lipid metabolism":
[	("00061", "Fatty acid biosynthesis"),
	("00062", "Fatty acid elongation"),	
	("00071", "Fatty acid degradation"),
	("00100", "Steroid biosynthesis"),	
	("00120", "Primary bile acid biosynthesis"),
	("00140", "Steroid hormone biosynthesis"),
	("00561", "Glycerolipid metabolism"),
	("00564", "Glycerophospholipid metabolism"),
	("00565", "Ether lipid metabolism"),
	("00600", "Sphingolipid metabolism"),
	("00590", "Arachidonic acid metabolism"),
	("00591", "Linoleic acid metabolism"),
	("00592", "alpha-Linolenic acid metabolism"),
	("01040", "Biosynthesis of unsaturated fatty acids")
],
"Nucleotide metabolism":
[
	("00230", "Purine metabolism"),
	("00240", "Pyrimidine metabolism")
],
"Amino acid metabolism":
[
	("00250", "Alanine, aspartate and glutamate metabolism"),
	("00260", "Glycine, serine and threonine metabolism"),
	("00270", "Cysteine and methionine metabolism"),
	("00280", "Valine, leucine and isoleucine degradation"),
	("00290", "Valine, leucine and isoleucine biosynthesis"),
	("00310", "Lysine degradation"),
	("00220", "Arginine biosynthesis"),
	("00330", "Arginine and proline metabolism"),
	("00340", "Histidine metabolism"),
	("00350", "Tyrosine metabolism"),
	("00360", "Phenylalanine metabolism"),
	("00380", "Tryptophan metabolism"),
	("00400", "Phenylalanine, tyrosine and tryptophan biosynthesis")
],
"Metabolism of other amino acids":
[
	("00410", "beta-Alanine metabolism"),
	("00430", "Taurine and hypotaurine metabolism"),
	("00440", "Phosphonate and phosphinate metabolism"),
	("00450", "Selenocompound metabolism"),
	("00470", "D-Amino acid metabolism"),
	("00480", "Glutathione metabolism")
],
"Glycan biosynthesis and metabolism":
[
	("00510", "N-Glycan biosynthesis"),
	("00513", "Various types of N-glycan biosynthesis"),
	("00512", "Mucin type O-glycan biosynthesis"),
	("00515", "Mannose type O-glycan biosynthesis"),
	("00514", "Other types of O-glycan biosynthesis"),
	("00532", "Glycosaminoglycan biosynthesis - chondroitin sulfate / dermatan sulfate"),
	("00534", "Glycosaminoglycan biosynthesis - heparan sulfate / heparin"),
	("00533", "Glycosaminoglycan biosynthesis - keratan sulfate"),
	("00531", "Glycosaminoglycan degradation"),
	("00563", "Glycosylphosphatidylinositol (GPI)-anchor biosynthesis"),
	("00601", "Glycosphingolipid biosynthesis - lacto and neolacto series"),
	("00603", "Glycosphingolipid biosynthesis - globo and isoglobo series"),
	("00604", "Glycosphingolipid biosynthesis - ganglio series"),
	("00511", "Other glycan degradation")
],
"Metabolism of cofactors and vitamins":
[
	("00730", "Thiamine metabolism"),
	("00740", "Riboflavin metabolism"),
	("00750", "Vitamin B6 metabolism"),
	("00760", "Nicotinate and nicotinamide metabolism"),
	("00770", "Pantothenate and CoA biosynthesis"),
	("00780", "Biotin metabolism"),
	("00785", "Lipoic acid metabolism"),
	("00790", "Folate biosynthesis"),
	("00670", "One carbon pool by folate"),
	("00830", "Retinol metabolism"),
	("00860", "Porphyrin metabolism"),
	("00130", "Ubiquinone and other terpenoid-quinone biosynthesis")
],
"Metabolism of terpenoids and polyketides":
[
	("00900", "Terpenoid backbone biosynthesis")
],
"Biosynthesis of other secondary metabolites":
[
	("00232", "Caffeine metabolism"),
	("00524", "Neomycin, kanamycin and gentamicin biosynthesis")
],
"Xenobiotics biodegradation and metabolism":
[
	("00980", "Metabolism of xenobiotics by cytochrome P450"),
	("00982", "Drug metabolism - cytochrome P450"),
	("00983", "Drug metabolism - other enzymes")
]
}

In [4]:
organismDict = {"hsa": "Homo sapiens (human)"}

In [5]:
outputDirectory = "/Users/boldi/Desktop/pw/"
dataDirectory = "../../LaTeX/Data/KEGG-Pathways/"

In [6]:
for organism in organismDict.keys():
    for superpathway in superpathwayDict.keys():
        safeSuperpathway = makesafe(superpathway)
        H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML(
            [os.path.join(dataDirectory, organism, f"{organism}{x[0]}-sbml.xml") for x in superpathwayDict[superpathway]])
        dd=hyper_cardon_crochemore(H, proxy=sr2r)
        
        groupedReactions = mapKeys(dd, prefOnly="rn")
        r2gr = {}
        r2grall = {}
        for i, reactions in enumerate(groupedReactions):
            for reaction in reactions:
                r2gr[reaction] = i
                r2grall[reaction] = reactions
                
        groupedCompounds = mapKeys(dd, prefOnly="C")
        c2gc = {}
        c2gcall = {}
        for i, compounds in enumerate(groupedCompounds):
            for compound in compounds:
                c2gc[compound] = i + len(groupedReactions)
                c2gcall[compound] = compounds
                
        writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "R", 
              f"{organismDict[organism]}, {superpathway}, Reactions group",
              onlyNonTrivial=True, prefOnly="r", cutPref="rn")
        index = [(f"{safeSuperpathway}R-cont-0.html", "Reaction groups")]
        writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "C", 
              f"{organismDict[organism]}, {superpathway}, Compound group",
              onlyNonTrivial=True, prefOnly="C", cutPref="")
        index += [(f"{safeSuperpathway}C-cont-0.html", "Compound groups")]
        
        index += writeKEGGpdf(outputDirectory, organism, superpathway, superpathwayDict, r2gr, r2grall, c2gc, c2gcall)
        
        writeIndex(os.path.join(outputDirectory, organism, safeSuperpathway), index, 
           f"{organismDict[organism]}, {superpathway}")
        
        with open(os.path.join(outputDirectory, organism, safeSuperpathway, "data.pkl"), "wb") as file:
            pickle.dump([dd, sr2r, r2ss, rid2rname, mid2mname], file)
    
    writeIndex(os.path.join(outputDirectory, organism), 
           [("{}/index.html".format(makesafe(k)), k) for k in superpathwayDict.keys()],
           f"{organismDict[organism]}")
    
writeIndex(os.path.join(outputDirectory), 
          [(f"{k}/index.html", v) for k,v in organismDict.items()],
           "Pathways")

/Users/boldi/.local/share/virtualenvs/qf-ZyPGaJ2v/lib/python3.9/site-packages/hypernetx/classes/hypergraph.py:968: UserWarning: Cannot add edge. Edge already in hypergraph
  warnings.warn("Cannot add edge. Edge already in hypergraph")


In [12]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import requests

In [75]:
basename = "listA"

with open(f"/Users/boldi/Desktop/{basename}.txt") as f:
    compounds = f.read().splitlines()

with open(f"/Users/boldi/Desktop/{basename}-complete.csv", "w") as f:
    for compound in compounds:
        if len(compound.strip()) == 0:
            continue
        print("Looking for ", compound)
        url = "https://www.kegg.jp/kegg-bin/search_ligand?query=" + quote_plus(compound) + "&column=entry%2Bname%2Bformula&DATABASE=compound"
        #print("First request: ", url)
        response = requests.get(url, allow_redirects=False)
        url = "https://www.kegg.jp/kegg-bin/" + response.headers["Location"]
        #print("Second request: ", url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        tds = [tds for tds in soup.find_all("td", {"class": "data1"})]
        cd = []
        d = {}
        for i in range(len(tds)//5):
            compound_name = tds[5*i+1].getText()
            name_td = tds[5*i+3].getText(strip=True, separator="|")
            cd += [compound_name]
            d[compound_name]=name_td
        if len(cd) > 0:
            f.write(f"{cd[0]},\"{d[cd[0]]}\",\"{compound}\"\n")
        else:
            f.write(f"?,?,\"{compound}\"\n")
        f.flush()

Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605294114916&show=entry,name,formula,structure&q=coenzyme%20A
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605297116373&show=entry,name,formula,structure&q=nicotinate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605300117715&show=entry,name,formula,structure&q=prephenate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605302117967&show=entry,name,formula,structure&q=reduced%20ferredoxin%20%5biron%2dsulfur%5d%20cluster
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605305118193&show=entry,name,formula,structure&q=glutathione%20disulfide
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605308118400&show=entry,name,formula,structure&q=dTMP
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=1682605311118821&show=entry,name,

Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054336596&show=entry,name,formula,structure&q=%7bbeta%2dD%2dGlcNAc%2d%281%2d%3e4%29%2d3%2dO%2d%5bL%2dAla%2dgamma%2dD%2diGln%2d%286%2dN%2dbeta%2dD%2dAsn%29%2dL%2dLys%2dD%2dAla%2dD%2dAla%5d%2dMur2Ac%7d%2dPP%2dUnd
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054367014&show=entry,name,formula,structure&q=presqualene%20diphosphate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054387383&show=entry,name,formula,structure&q=choloyl%2dCoA
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054418888&show=entry,name,formula,structure&q=mycofactocinol
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054449297&show=entry,name,formula,structure&q=acetoacetyl%2d%5bacp%5d
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=16826054469652&show=entry,name,formula,struct

Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260556727378&show=entry,name,formula,structure&q=catechol
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260557027689&show=entry,name,formula,structure&q=n%204%2ddeoxy%2dbeta%2dD%2dgluc%2d4%2denuronosyl%2d%281%2c3%29%2dN%2dacetyl%2dD%2dgalactosamine%204%2dsulfate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260557327912&show=entry,name,formula,structure&q=chondroitin%20sulfate%20A%28n%2d1%29
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260557528218&show=entry,name,formula,structure&q=4%2ddeoxy%2dbeta%2dD%2dgluc%2d4%2denuronosyl%2d%281%2c3%29%2dN%2dacetyl%2dD%2dgalactosamine
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260557828496&show=entry,name,formula,structure&q=chondroitin%20sulfate%20C%28n%2d1%29
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid

Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260568645840&show=entry,name,formula,structure&q=%288S%29%2d8%2damino%2d7%2doxononanoate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260568946114&show=entry,name,formula,structure&q=keto%2dD%2dtagatose%206%2dphosphate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260569146396&show=entry,name,formula,structure&q=3%2dmethylglutaconyl%2dCoA
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260569446655&show=entry,name,formula,structure&q=oxalyl%2dCoA
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260569646840&show=entry,name,formula,structure&q=phosphatidylethanolamine
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pid=168260569947049&show=entry,name,formula,structure&q=diethylphosphate
Second request:  https://www.kegg.jp/kegg-bin/search_result?db=compound&pi